In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2

In [2]:
# load class_names
labels = np.load("class_names.npy", allow_pickle=True).item()

# load attributes
attributes = np.load("attributes.npy")
print(attributes.shape)
attributes_dict = {i + 1: row for i, row in enumerate(attributes)}
# load training data
train_df = pd.read_csv("train_images.csv")
# add attributes to data
train_df["attributes"] = train_df["label"].map(attributes_dict)
train_df.head()

(200, 312)


,image_path,label,attributes
0,/train_images/1.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."
1,/train_images/2.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."
2,/train_images/3.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."
3,/train_images/4.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."
4,/train_images/5.jpg,1,"[0.010638400403539122, 0.010638400403539122, 0..."


In [3]:
# Initialize empty lists for images (X) and labels (y)
X = []
y = []

# Iterate over rows in the DataFrame
for _, row in train_df.iterrows():
    # Read the image
    im_path = row['image_path']
    image = cv2.imread(f'train_images{im_path}')
    
    # Resize the image to 224x224
    image_resized = cv2.resize(image, (224, 224))
    
    # Append to the list of images and labels
    X.append(image_resized)

# Convert lists to numpy arrays (optional)
X = np.array(X)
y = train_df['label']

In [9]:
# Convert y to categorical
y_c = to_categorical(y.values-1)
X_train, X_test, y_train, y_test = train_test_split(X, y_c, test_size=0.2, random_state=0)
# use one-hot encoding for labels
print(y_train.shape)

(3140, 200)


In [10]:
# Initialize the EfficientNetV2 model with default settings
base_model = EfficientNetV2B2(
    include_top=False,       # Exclude the classification head
    weights="imagenet",      # Use pre-trained weights from ImageNet
    input_shape=(224, 224, 3)  # Input image size (H, W, C)
)

                                                                                                  
 block4a_bn (BatchNormaliza  (None, 14, 14, 224)          896       ['block4a_dwconv2[0][0]']     
 tion)                                                                                            
                                                                                                  
 block4a_activation (Activa  (None, 14, 14, 224)          0         ['block4a_bn[0][0]']          
 tion)                                                                                            
                                                                                                  
 block4a_se_squeeze (Global  (None, 224)                  0         ['block4a_activation[0][0]']  
 AveragePooling2D)                                                                                
                                                                                                  
 block4a_s

 block4c_activation (Activa  (None, 14, 14, 416)          0         ['block4c_bn[0][0]']          
 tion)                                                                                            
                                                                                                  
 block4c_se_squeeze (Global  (None, 416)                  0         ['block4c_activation[0][0]']  
 AveragePooling2D)                                                                                
                                                                                                  
 block4c_se_reshape (Reshap  (None, 1, 1, 416)            0         ['block4c_se_squeeze[0][0]']  
 e)                                                                                               
                                                                                                  
 block4c_se_reduce (Conv2D)  (None, 1, 1, 26)             10842     ['block4c_se_reshape[0][0]']  
          

                                                                                                  
 block5a_activation (Activa  (None, 14, 14, 624)          0         ['block5a_bn[0][0]']          
 tion)                                                                                            
                                                                                                  
 block5a_se_squeeze (Global  (None, 624)                  0         ['block5a_activation[0][0]']  
 AveragePooling2D)                                                                                
                                                                                                  
 block5a_se_reshape (Reshap  (None, 1, 1, 624)            0         ['block5a_se_squeeze[0][0]']  
 e)                                                                                               
                                                                                                  
 block5a_s

 block5c_se_squeeze (Global  (None, 720)                  0         ['block5c_activation[0][0]']  
 AveragePooling2D)                                                                                
                                                                                                  
 block5c_se_reshape (Reshap  (None, 1, 1, 720)            0         ['block5c_se_squeeze[0][0]']  
 e)                                                                                               
                                                                                                  
 block5c_se_reduce (Conv2D)  (None, 1, 1, 30)             21630     ['block5c_se_reshape[0][0]']  
                                                                                                  
 block5c_se_expand (Conv2D)  (None, 1, 1, 720)            22320     ['block5c_se_reduce[0][0]']   
                                                                                                  
 block5c_s

                                                                                                  
 block5e_se_squeeze (Global  (None, 720)                  0         ['block5e_activation[0][0]']  
 AveragePooling2D)                                                                                
                                                                                                  
 block5e_se_reshape (Reshap  (None, 1, 1, 720)            0         ['block5e_se_squeeze[0][0]']  
 e)                                                                                               
                                                                                                  
 block5e_se_reduce (Conv2D)  (None, 1, 1, 30)             21630     ['block5e_se_reshape[0][0]']  
                                                                                                  
 block5e_se_expand (Conv2D)  (None, 1, 1, 720)            22320     ['block5e_se_reduce[0][0]']   
          

 tion)                                                                                            
                                                                                                  
 block6a_se_squeeze (Global  (None, 720)                  0         ['block6a_activation[0][0]']  
 AveragePooling2D)                                                                                
                                                                                                  
 block6a_se_reshape (Reshap  (None, 1, 1, 720)            0         ['block6a_se_squeeze[0][0]']  
 e)                                                                                               
                                                                                                  
 block6a_se_reduce (Conv2D)  (None, 1, 1, 30)             21630     ['block6a_se_reshape[0][0]']  
                                                                                                  
 block6a_s

                                                                                                  
 block6c_se_reshape (Reshap  (None, 1, 1, 1248)           0         ['block6c_se_squeeze[0][0]']  
 e)                                                                                               
                                                                                                  
 block6c_se_reduce (Conv2D)  (None, 1, 1, 52)             64948     ['block6c_se_reshape[0][0]']  
                                                                                                  
 block6c_se_expand (Conv2D)  (None, 1, 1, 1248)           66144     ['block6c_se_reduce[0][0]']   
                                                                                                  
 block6c_se_excite (Multipl  (None, 7, 7, 1248)           0         ['block6c_activation[0][0]',  
 y)                                                                  'block6c_se_expand[0][0]']   
          

 AveragePooling2D)                                                                                
                                                                                                  
 block6e_se_reshape (Reshap  (None, 1, 1, 1248)           0         ['block6e_se_squeeze[0][0]']  
 e)                                                                                               
                                                                                                  
 block6e_se_reduce (Conv2D)  (None, 1, 1, 52)             64948     ['block6e_se_reshape[0][0]']  
                                                                                                  
 block6e_se_expand (Conv2D)  (None, 1, 1, 1248)           66144     ['block6e_se_reduce[0][0]']   
                                                                                                  
 block6e_se_excite (Multipl  (None, 7, 7, 1248)           0         ['block6e_activation[0][0]',  
 y)       

 block6g_se_squeeze (Global  (None, 1248)                 0         ['block6g_activation[0][0]']  
 AveragePooling2D)                                                                                
                                                                                                  
 block6g_se_reshape (Reshap  (None, 1, 1, 1248)           0         ['block6g_se_squeeze[0][0]']  
 e)                                                                                               
                                                                                                  
 block6g_se_reduce (Conv2D)  (None, 1, 1, 52)             64948     ['block6g_se_reshape[0][0]']  
                                                                                                  
 block6g_se_expand (Conv2D)  (None, 1, 1, 1248)           66144     ['block6g_se_reduce[0][0]']   
                                                                                                  
 block6g_s

                                                                                                  
 block6i_se_squeeze (Global  (None, 1248)                 0         ['block6i_activation[0][0]']  
 AveragePooling2D)                                                                                
                                                                                                  
 block6i_se_reshape (Reshap  (None, 1, 1, 1248)           0         ['block6i_se_squeeze[0][0]']  
 e)                                                                                               
                                                                                                  
 block6i_se_reduce (Conv2D)  (None, 1, 1, 52)             64948     ['block6i_se_reshape[0][0]']  
                                                                                                  
 block6i_se_expand (Conv2D)  (None, 1, 1, 1248)           66144     ['block6i_se_reduce[0][0]']   
          

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# Freeze the base model
base_model.trainable = False

# Add a custom classification head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation="relu"),
    layers.Dropout(0.5),
    Dense(200, activation='softmax')
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Fit the data
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=20,
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Epoch 1/20
99/99 [==============================] - 92s 927ms/step - loss: 3.0961 - accuracy: 0.2685 - val_loss: 2.6802 - val_accuracy: 0.3651
Epoch 2/20
99/99 [==============================] - 93s 940ms/step - loss: 2.5034 - accuracy: 0.3694 - val_loss: 2.3621 - val_accuracy: 0.4046
Epoch 3/20
37/99 [==========>...................] - ETA: 47s - loss: 2.2192 - accuracy: 0.4282

In [20]:
model.save("efficientnetv2B2_model.h5")

C:\Users\brake\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# Unfreeze a few layers at a time
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model after unfreezing layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

# Continue training
history_improved = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=5,
    callbacks=[early_stopping]
)

Epoch 1/5
99/99 [==============================] - 96s 901ms/step - loss: 0.7788 - accuracy: 0.8924 - val_loss: 1.7855 - val_accuracy: 0.5471
Epoch 2/5
99/99 [==============================] - 87s 877ms/step - loss: 0.7699 - accuracy: 0.8914 - val_loss: 1.7799 - val_accuracy: 0.5471
Epoch 3/5
99/99 [==============================] - 87s 876ms/step - loss: 0.7624 - accuracy: 0.8946 - val_loss: 1.7966 - val_accuracy: 0.5445
Epoch 4/5
99/99 [==============================] - 86s 870ms/step - loss: 0.7601 - accuracy: 0.8978 - val_loss: 1.7911 - val_accuracy: 0.5471
Epoch 5/5
99/99 [==============================] - 86s 867ms/step - loss: 0.7517 - accuracy: 0.9013 - val_loss: 1.7754 - val_accuracy: 0.5534


In [21]:
test_df = pd.read_csv("test_images_path.csv")

test_images = []
# Iterate over rows in the DataFrame
for _, row in test_df.iterrows():
    # Read the image
    im_path = row['image_path']
    image = cv2.imread(f'test_images{im_path}')
    
    # Resize the image to 400x400
    image_resized = cv2.resize(image, (224, 224))
    
    # Append to the list of images and labels
    test_images.append(image_resized)


predictions = model.predict(np.array(test_images))
predicted_labels = np.argmax(predictions, axis = 1)

submission_df = pd.DataFrame({
    'id': test_df['id'],
    'label': predicted_labels + 1
})
submission_df.to_csv('submission.csv', index=False)

125/125 [==============================] - 88s 692ms/step
